In [10]:
import pandas as pd
import networkx as nx
from community import community_louvain
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from typing import Dict, Tuple


data: pd.DataFrame = pd.read_csv('all_collected_data.csv', encoding="EUC-KR")
edges: pd.DataFrame = data[["출발 행정동 코드", "도착 행정동 코드", "이동인구(합)"]]

G = nx.Graph()

for index, row in tqdm(edges.iterrows(), total=edges.shape[0], desc="Creating Graph"):
    if pd.notna(row["이동인구(합)"]):
        G.add_edge(
            row["출발 행정동 코드"], row["도착 행정동 코드"], weight=row["이동인구(합)"]
        )


Creating Graph: 100%|██████████| 3225/3225 [00:00<00:00, 43095.46it/s]


In [11]:
resolution_values = [0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1, 1.5, 2, 3]

for resolution in resolution_values:
    partition = community_louvain.best_partition(G, weight="weight", resolution=resolution)

    # Generate positions for nodes for plotting
    pos = nx.spring_layout(G)

    # Save results to a text file named by the resolution value
    file_name = f"community_results_{resolution:.2f}.txt"  # Formatting resolution to two decimals
    with open(file_name, "w") as file:
        for node, comm in partition.items():
            file.write(f"{node} : {comm}\n")

    print(f"Community detection result for resolution {resolution} saved to {file_name}")


Community detection result for resolution 0.05 saved to community_results_0.05.txt
Community detection result for resolution 0.1 saved to community_results_0.10.txt
Community detection result for resolution 0.3 saved to community_results_0.30.txt
Community detection result for resolution 0.5 saved to community_results_0.50.txt
Community detection result for resolution 0.7 saved to community_results_0.70.txt
Community detection result for resolution 0.9 saved to community_results_0.90.txt
Community detection result for resolution 1 saved to community_results_1.00.txt
Community detection result for resolution 1.5 saved to community_results_1.50.txt
Community detection result for resolution 2 saved to community_results_2.00.txt
Community detection result for resolution 3 saved to community_results_3.00.txt


In [12]:
import pandas as pd

def load_partition(file_path: str) -> dict:
    partition = {}
    with open(file_path, 'r') as file:
        for line in file:
            parts = line.strip().split(' : ')
            if len(parts) == 2:
                key = float(parts[0])
                value = int(parts[1])
                partition[key] = value
    return partition

partition_file_path = 'community_results_1.50.txt'
excel_file_path = 'data/서울생활이동데이터_자치구코드_20210907.xlsx'

partition = load_partition(partition_file_path)

df = pd.read_excel(excel_file_path)
community_names = {}

for key, community in partition.items():
    match_row = df[df['시군구'] == key]
    if not match_row.empty:
        name_value = match_row['name'].values[0]
        if community in community_names:
            community_names[community].append(name_value)
        else:
            community_names[community] = [name_value]

for community, names in community_names.items():
    print(f"Community {community}: {names}")


Community 0: ['종로구', '중구']
Community 5: ['용산구', '평택시']
Community 3: ['성동구']
Community 6: ['광진구']
Community 12: ['동대문구', '중랑구', '구리시', '남양주시', '가평군', '양평군']
Community 8: ['성북구', '강북구']
Community 2: ['도봉구', '노원구', '의정부시', '동두천시', '양주시', '포천시', '연천군']
Community 11: ['은평구', '덕양구']
Community 13: ['서대문구', '마포구', '일산동구', '일산서구', '파주시']
Community 15: ['양천구', '강서구', '중구', '계양구', '서구', '강화군', '김포시']
Community 4: ['구로구', '금천구', '동구', '남구', '남동구', '부평구', '옹진군', '만안구', '부천시', '광명시', '상록구', '단원구', '시흥시', '군포시']
Community 16: ['영등포구', '연수구']
Community 7: ['동작구', '관악구']
Community 1: ['서초구', '장안구', '권선구', '팔달구', '동안구', '과천시', '의왕시', '용인시', '안성시', '화성시']
Community 9: ['강남구', '영통구', '분당구', '오산시', '용인시', '용인시']
Community 10: ['송파구', '수정구', '중원구', '이천시', '광주시', '여주시']
Community 14: ['강동구', '하남시']


In [14]:
import pandas as pd
import os

def load_partition(file_path):
    partition = {}
    with open(file_path, 'r', encoding='utf-8') as file:  # 파일을 UTF-8로 열기
        for line in file:
            parts = line.strip().split(' : ')
            if len(parts) == 2:
                key = float(parts[0])
                value = int(parts[1])
                partition[key] = value
    return partition

base_path = r"C:\Users\kctco\Documents\Code\seoul-communitydetection\2023\생활이동_자치구_2023"
excel_file_path = 'data/서울생활이동데이터_자치구코드_20210907.xlsx'

df = pd.read_excel(excel_file_path)

for resolution in resolution_values:
    partition_file_path = f'community_results_{resolution:.2f}.txt'
    partition = load_partition(partition_file_path)
    community_names = {}

    for key, community in partition.items():
        match_row = df[df['시군구'] == key]
        if not match_row.empty:
            name_value = match_row['name'].values[0]
            if community in community_names:
                community_names[community].append(name_value)
            else:
                community_names[community] = [name_value]

    results_file_name = f"community_results_{resolution:.2f}_kor.txt"
    with open(results_file_name, 'w', encoding='EUC-KR') as file:
        for community, names in community_names.items():
            file.write(f"Community {community}: {', '.join(names)}\n")

    print(f"Data for resolution {resolution:.2f} saved to {results_file_name}")


Data for resolution 0.05 saved to community_results_0.05_kor.txt
Data for resolution 0.10 saved to community_results_0.10_kor.txt
Data for resolution 0.30 saved to community_results_0.30_kor.txt
Data for resolution 0.50 saved to community_results_0.50_kor.txt
Data for resolution 0.70 saved to community_results_0.70_kor.txt
Data for resolution 0.90 saved to community_results_0.90_kor.txt
Data for resolution 1.00 saved to community_results_1.00_kor.txt
Data for resolution 1.50 saved to community_results_1.50_kor.txt
Data for resolution 2.00 saved to community_results_2.00_kor.txt
Data for resolution 3.00 saved to community_results_3.00_kor.txt
